In [32]:
# Load necessary libraries
import pandas as pd 

# Data Ingestion

In [33]:
cards_csv = pd.read_csv('../dataset/cards.csv', sep=";")
prices_csv = pd.read_csv('../dataset/cardPrices.csv', sep=",")

print(f"Cards Dataset length = {len(cards_csv)}")
print(f"Prices Dataset length = {len(prices_csv)} \n")

print(f"Cards Columns Num = {cards_csv.shape[1]}")
print(f"Price Columns Num = {prices_csv.shape[1]}\n")

# number of unique dataset instances by uuid
card_unique = cards_csv['uuid'].nunique()
price_unique = prices_csv['uuid'].nunique()

print(f"Cards length by UUID = {card_unique}")
print(f"Prices length by UUID= {price_unique}\n")

dates = prices_csv['date'].nunique()
date = prices_csv['date'][0]
print(f"Num of unique dates = {dates}")
print(f"Price Date = {date}")

#print(cards_csv.nunique())
#print(prices_csv.nunique())

Cards Dataset length = 97145
Prices Dataset length = 558079 

Cards Columns Num = 25
Price Columns Num = 8

Cards length by UUID = 97145
Prices length by UUID= 91302

Num of unique dates = 1
Price Date = 2024-09-20


In [34]:
cards = cards_csv
prices = prices_csv

# Drop columns that are duplicates
cards = cards.drop(columns=['finishes'])
cards = cards.drop(columns=['hasFoil'])
cards = cards.drop(columns=['hasNonFoil'])
cards = cards.drop(columns=['sourceProducts'])

# Drop constant columns
prices = prices[prices['currency'] == 'USD']
prices = prices.drop(columns=['currency'])
prices = prices.drop(columns=['date'])

# Standardize colors and colorIdentity
def unique_colors(value):
    if pd.isna(value):
        return None  # Return None for NaN
    # Split the value, standardize, and get unique characters
    characters = set(', '.join(sorted(value.split(', '))).replace(', ', ''))
    return ''.join(sorted(characters))

# Apply the function to the columns
cards['colors'] = cards['colors'].apply(unique_colors)
cards['colorIdentity'] = cards['colorIdentity'].apply(unique_colors)

# replace NaN values with False
cards.loc[cards['isReprint'].isna(), "isReprint"] = False
# Map True to 1 and False to 0
cards['isReprint'] = cards['isReprint'].astype(int)

In [35]:
# cards2 is going to be used in secondary
cards2 = cards.copy()

cards2['colors'] = cards2['colors'].fillna('C')
cards2['colorIdentity'] = cards2['colorIdentity'].fillna('C')

cards2['originalType'] = cards2['originalType'].fillna('None')
cards2['power'] = cards2['power'].fillna('None')
cards2['supertypes'] = cards2['supertypes'].fillna('None')
cards2['toughness'] = cards2['toughness'].fillna('None')

In [36]:
# perform inner join based on UUID, then drop null values
primary = pd.merge(prices, cards, on="uuid")
primary = primary.dropna().reset_index(drop=True)
print(f"Primary dataset by uuid (cards with prices) size: {len(primary)}")


secondary = pd.merge(prices, cards2, on="uuid")
secondary = secondary.dropna().reset_index(drop=True)
print(f"Seconday dataset by uuid (cards with prices) size: {len(secondary)}")

Primary dataset by uuid (cards with prices) size: 17628
Seconday dataset by uuid (cards with prices) size: 269807


In [38]:
primary.to_csv('../dataset/primary.csv', index = False)
secondary.to_csv('../dataset/secondary.csv', index = False)